# Load data

In [118]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

In [119]:
df = pd.read_csv(r'..\data\passing-grade.csv')
dummy = pd.read_csv(r"..\data\tryout_data.csv")

Dataset ini diambil dari dua sumber. Pertama, dataset utama (df) bersumber dari kaggle ("https://www.kaggle.com/datasets/rezkyyayang/passing-grade-utbk-in-science-major/data"). Kedua, dataset tambahan (dummy) yang didapatkan setelah proses ekstraksi data dari website (sc : "https://hasilto.bimbelssc.com/storage/ponorogo/intipa/data/TO_SNBT_JANUARI.html").

# EDA

In [120]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   NO          500 non-null    int64  
 1   PTN         500 non-null    object 
 2   KODE PRODI  500 non-null    int64  
 3   NAMA PRODI  500 non-null    object 
 4   RATAAN      500 non-null    float64
 5   S.BAKU      500 non-null    float64
 6   MIN         500 non-null    float64
 7   MAX         500 non-null    float64
dtypes: float64(4), int64(2), object(2)
memory usage: 31.4+ KB


Dataset utama terdiri dari 500 baris (entries) dan 8 kolom, yaitu:
- NO: Tipe data int64, 500 data non-null.
- PTN: Tipe data object (string), 500 data non-null.
- KODE_PRODI: Tipe data int64, 500 data non-null.
- NAMA_PRODI: Tipe data object (string), 500 data non-null.
- RATAAN: Tipe data float64, 500 data non-null.
- SBAKU: Tipe data float64, 500 data non-null.
- MIN: Tipe data float64, 500 data non-null.
- MAX: Tipe data float64, 500 data non-null.

In [121]:
print(df['PTN'].unique())

['UNIVERSITAS INDONESIA' 'UNIVERSITAS AIRLANGGA' 'UNIVERSITAS PADJADJARAN'
 'UNIVERSITAS GADJAH MADA' 'UNIVERSITAS DIPONEGORO'
 'INSTITUT TEKNOLOGI BANDUNG' 'UNIVERSITAS BRAWIJAYA'
 'UNIVERSITAS SEBELAS MARET' 'UNIVERSITAS JENDERAL SOEDIRMAN'
 'UNIVERSITAS JEMBER' 'UNIVERSITAS UDAYANA'
 'INSTITUT TEKNOLOGI SEPULUH NOPEMBER' 'UNIVERSITAS SRIWIJAYA'
 'UPN "VETERAN" JAKARTA' 'UNIVERSITAS SUMATERA UTARA'
 'UNIVERSITAS ISLAM NEGERI MALANG' 'UNIVERSITAS ANDALAS'
 'UNIVERSITAS HASANUDDIN' 'UNIVERSITAS ISLAM NEGERI JAKARTA'
 'UNIVERSITAS MATARAM' 'INSTITUT PERTANIAN BOGOR' 'UNIVERSITAS LAMPUNG'
 'UNIVERSITAS SYIAH KUALA' 'UNIVERSITAS RIAU'
 'UNIVERSITAS PENDIDIKAN GANESHA' 'UNIVERSITAS MULAWARMAN'
 'UNIVERSITAS LAMBUNG MANGKURAT' 'UNIVERSITAS TANJUNGPURA'
 'UNIVERSITAS JAMBI' 'UNIVERSITAS SAM RATULANGI'
 'UNIVERSITAS NEGERI YOGYAKARTA' 'UNIVERSITAS NEGERI JAKARTA'
 'UNIVERSITAS CENDERAWASIH' 'UNIVERSITAS BENGKULU'
 'UNIVERSITAS MALIKUSSALEH' 'UNIVERSITAS PENDIDIKAN INDONESIA'
 'UNIVERSITAS NUS

pada dataset ini memiliki 50 perguruan tinggi yang daftarnya bisa dilihat pada output di atas.

In [122]:
rata2_per_PTN = df.groupby('PTN')['RATAAN'].mean()
urutan_PTN = rata2_per_PTN.sort_values(ascending=False)
df = df.set_index('PTN').loc[urutan_PTN.index].reset_index()

Mengelompokkan data berdasarkan kolom 'PTN' dan menghitung rata-rata nilai dari kolom 'RATAAN' untuk setiap PTN. Ini dilakukan untuk melihat PTN mana yang memiliki rata-rata nilai tertinggi dan terendah.

In [123]:
batasMean = df['RATAAN'].min()
batasSBaku = df['S.BAKU'].max()
batasMin = df['MIN'].min()
batasMax = df['MAX'].min()

print(f"nilai rataan terkecil : {batasMean}")
print(f"nilai S.baku terbesar : {batasSBaku}")
print(f"nilai MIN terkecil : {batasMin}")
print(f"nilai MAX terkecil : {batasMin}")

nilai rataan terkecil : 594.6
nilai S.baku terbesar : 29.58
nilai MIN terkecil : 581.92
nilai MAX terkecil : 581.92


Berikutnya dicari nilai minimum dari kolom "RATAAN", "MIN", dan "MAX". kemudian nilai maksimum dari kolom "S.BAKU" yang nantinya akan digunakan sebagai batasan 

# Prepration data dummy

Pada tahap ini kolom "participant_no", "name", "status_kelulusan", dan "timestamp" akan dihilangkan karena tidak dibutuhkan dalam klasifikasi

In [124]:
dummy = dummy.drop(columns=['participant_no', 'name', 'status_kelulusan', 'timestamp'])
dummy.head()

,pu,ppu,kmbm,pk,lit_ind,lit_ing,pm,total
0,"855,84","879,66","818,52","487,19","708,43","909,85","741,38","771,55"
1,"842,78","747,67","797,22","410,43","602,02","909,85","1000,00","758,57"
2,"787,79","545,34","783,33","593,05","775,49","742,14","1000,00","746,73"
3,"793,78","629,57","784,26","631,43","646,48","909,85","825,43","745,83"
4,"777,99","611,33","669,44","584,38","725,20","909,85","895,11","739,04"


Selanjutnya, akan dilakukan beberapa tahap preprocessing, yaitu:
1. mengganti nilai nol yang ditandai dengan "X" menjadi 0. 
2. kemudian menghapus spasi dan mengganti tanda koma menjadi titik. 
3. Selanjutnya mengubah tipe data menjadi numerik agar dapat dilakukan feature engineering.
4. mengisi missing value dengan nilai rata-rata karena metode ini mempertahankan distribusi data dan tidak mempengaruhi ukuran sampel secara signifikan. 
5. Selanjutnya menghapus data yang terduplikasi.
6. Akan dibuat kolom baru yang berisi nilai rata-rata, simpangan baku, total, terkecil dan terbesar pada setiap baris data untuk disesuaikan pada data utama.

In [125]:
# 1. Ganti 'X' dengan '0'
dummy = dummy.replace('X', 0)

# 2. Bersihkan data: hapus spasi & ganti koma dengan titik
dummy = dummy.applymap(lambda x: str(x).replace(' ', '').replace(',', '.'))

# 3. Konversi ke float per kolom (lebih aman)
for col in dummy.columns:
    dummy[col] = pd.to_numeric(dummy[col], errors='coerce')

# 4. Handle missing value
dummy = dummy.fillna(dummy.mean())

# 5. Handle duplicate
dummy = dummy.drop_duplicates()

# 6. Feature engineering
dummy['RATAAN'] = dummy[['pu', 'ppu', 'kmbm', 'pk', 'lit_ind', 'lit_ing', 'pm']].mean(axis=1)
dummy['S.BAKU'] = dummy[['pu', 'ppu', 'kmbm', 'pk', 'lit_ind', 'lit_ing', 'pm']].std(axis=1)
dummy['MIN'] = dummy[['pu', 'ppu', 'kmbm', 'pk', 'lit_ind', 'lit_ing', 'pm']].min(axis=1)
dummy['MAX'] = dummy[['pu', 'ppu', 'kmbm', 'pk', 'lit_ind', 'lit_ing', 'pm']].max(axis=1)

C:\Users\hp\AppData\Local\Temp\ipykernel_10028\719988935.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dummy = dummy.applymap(lambda x: str(x).replace(' ', '').replace(',', '.'))


Pada tahap selanjutnya melibatkan penyesuaian nilai dalam beberapa kolom ('RATAAN', 'MIN', 'MAX', dan 'S.BAKU') berdasarkan ambang batas yang telah ditentukan, dengan tujuan menormalkan atau membersihkan data. Hal ini dilakukan untuk menghindari nilai yang terlalu rendah dibandingkan ambang batas, yang dianggap tidak realistis atau outlier. Dengan mengganti nilai tersebut dengan angka acak dalam rentang yang ditentukan, data menjadi lebih konsisten dan sesuai dengan ekspektasi analisis berikutnya. Rentang hingga 1000 dipilih sebagai batas atas yang wajar berdasarkan konteks data. Pada kolom 'S.BAKU' (standar deviasi) seharusnya tidak melebihi nilai tertentu yang telah ditentukan (29.58) untuk menjaga konsistensi variabilitas data. Nilai acak antara 0 dan batas dipilih untuk menormalisasi standar deviasi yang terlalu tinggi, yang bisa mengindikasikan noise atau kesalahan pengukuran. Selanjutnya, pembulatan dilakukan untuk menyederhanakan angka-angka dalam data, meningkatkan keterbacaan, dan mengurangi presisi yang tidak perlu.

In [126]:
import numpy as np

# Ambang batas
batas = {
    'RATAAN': batasMean,
    'MIN': batasMin,
    'MAX': batasMax,
    'S.BAKU':batasSBaku
}

# Ganti nilai yang kurang dari batas dengan nilai random dari (nilai min di kolom sampai 1000)
for kolom in ['RATAAN', 'MIN', 'MAX']:
    nilai_min = dummy[kolom].min()
    
    # Mask untuk nilai yang kurang dari batas
    mask = dummy[kolom] < batas[kolom]
    
    # Buat nilai random untuk posisi yang perlu diganti
    dummy.loc[mask, kolom] = np.random.uniform(batas[kolom], 1000, size=mask.sum())

nilai_min = dummy["S.BAKU"].min()
# Mask untuk nilai yang kurang dari batas
mask = dummy["S.BAKU"] > batas["S.BAKU"]    
# Buat nilai random untuk posisi yang perlu diganti
dummy.loc[mask, "S.BAKU"] = np.random.uniform(0, batas["S.BAKU"], size=mask.sum())

dummy[['RATAAN', 'S.BAKU', 'MIN', 'MAX']] = dummy[['RATAAN', 'S.BAKU','MIN', 'MAX']].round(2)



In [127]:
dummy.describe()

,pu,ppu,kmbm,pk,lit_ind,lit_ing,pm,total,RATAAN,S.BAKU,MIN,MAX
count,3247.000000,3247.000000,3247.000000,3247.000000,3247.000000,3247.000000,3247.000000,3247.000000,3247.000000,3247.000000,3247.000000,3247.000000
mean,312.991745,328.843809,413.591706,189.290136,448.142707,466.885359,265.857167,346.523822,792.833184,14.848620,788.431571,792.292307
std,171.589505,155.305398,180.924047,106.563074,206.723778,274.911194,182.808438,133.637419,118.443444,8.475456,120.407393,103.964473
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.770000,594.600000,0.010000,582.010000,621.310000
25%,200.890000,230.555000,299.070000,128.760000,303.930000,229.035000,142.240000,249.915000,690.340000,7.560000,682.440000,704.400000
50%,312.590872,336.930000,426.850000,185.710000,440.950000,474.840000,233.480000,348.900000,790.340000,14.960000,786.900000,788.260000
75%,430.640000,437.660000,545.370000,250.100000,604.940000,703.350000,357.400000,444.530000,895.055000,22.160000,890.230000,871.495000
max,870.000000,950.000000,950.000000,650.000000,930.000000,1000.000000,1000.000000,771.550000,999.860000,29.510000,999.380000,1000.000000


In [142]:
df

,PTN,NO,KODE PRODI,NAMA PRODI,RATAAN,S.BAKU,MIN,MAX
0,UNIVERSITAS INDONESIA,1,3211015,PENDIDIKAN DOKTER,747.93,19.63,724.38,798.55
1,UNIVERSITAS INDONESIA,5,3211166,ILMU KOMPUTER,712.41,24.13,685.96,798.66
2,UNIVERSITAS INDONESIA,9,3211313,AKTUARIA,716.32,29.35,681.72,782.33
3,UNIVERSITAS INDONESIA,21,3211232,ARSITEKTUR INTERIOR,681.69,13.46,663.43,716.90
4,UNIVERSITAS INDONESIA,23,3211182,TEKNIK INDUSTRI,690.30,19.63,663.30,734.80
...,...,...,...,...,...,...,...,...
495,UNIVERSITAS NEGERI MALANG,488,3731226,TEKNIK INDUSTRI,597.91,13.35,583.19,636.45
496,UNIVERSITAS NEGERI MALANG,494,3731145,BIOLOGI,597.97,15.94,582.40,643.74
497,UNIVERSITAS NEGERI MALANG,498,3731184,ILMU KESEHATAN MASYARAKAT,598.79,17.46,582.05,687.01
498,UNIVERSITAS ISLAM NEGERI SUNAN GUNUNG DJATI,484,3361066,TEKNIK INFORMATIKA,603.88,16.75,583.54,653.39


# Modeling

Tahap ini melakukan modeling dengan random forest menggunakan data utama untuk memprediksi data dummy guna menciptakan data baru

In [128]:
X_train = df[['RATAAN', 'S.BAKU', 'MIN', 'MAX']]
y_train = df['PTN']
X_test = dummy[['RATAAN', 'S.BAKU', 'MIN', 'MAX']]

In [129]:
model = RandomForestClassifier(n_estimators=100, random_state=42)

model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [130]:
y_2 = model.predict(X_test)

dummy["PTN"] = y_2

membuat data baru yang berisi nantinya akan digabungkan dengan data utama

In [131]:
dummyBaru = dummy[["RATAAN", "S.BAKU", "MIN", "MAX", "PTN"]]
dummyBaru

,RATAAN,S.BAKU,MIN,MAX,PTN
0,771.55,0.50,722.91,909.85,UNIVERSITAS AIRLANGGA
1,758.57,17.34,970.97,1000.00,UNIVERSITAS AIRLANGGA
2,746.73,10.53,799.76,1000.00,UNIVERSITAS AIRLANGGA
3,745.83,4.16,629.57,909.85,UNIVERSITAS INDONESIA
4,739.04,8.24,584.38,909.85,UNIVERSITAS BRAWIJAYA
...,...,...,...,...,...
3244,626.20,12.14,691.04,752.88,UNIVERSITAS BRAWIJAYA
3245,879.33,12.12,759.24,795.35,UNIVERSITAS INDONESIA
3246,980.30,10.17,904.50,731.53,UNIVERSITAS JENDERAL SOEDIRMAN
3247,993.81,8.64,729.13,738.05,UNIVERSITAS JENDERAL SOEDIRMAN


menggabungkan data baru dan data utama

In [132]:
gabungan = pd.concat([df[["RATAAN", "S.BAKU", "MIN", "MAX", "PTN"]], dummyBaru], ignore_index=True)
gabungan

,RATAAN,S.BAKU,MIN,MAX,PTN
0,747.93,19.63,724.38,798.55,UNIVERSITAS INDONESIA
1,712.41,24.13,685.96,798.66,UNIVERSITAS INDONESIA
2,716.32,29.35,681.72,782.33,UNIVERSITAS INDONESIA
3,681.69,13.46,663.43,716.90,UNIVERSITAS INDONESIA
4,690.30,19.63,663.30,734.80,UNIVERSITAS INDONESIA
...,...,...,...,...,...
3742,626.20,12.14,691.04,752.88,UNIVERSITAS BRAWIJAYA
3743,879.33,12.12,759.24,795.35,UNIVERSITAS INDONESIA
3744,980.30,10.17,904.50,731.53,UNIVERSITAS JENDERAL SOEDIRMAN
3745,993.81,8.64,729.13,738.05,UNIVERSITAS JENDERAL SOEDIRMAN


melakukan preparation pada data baru, yaitu membagi data menjadi 80% data latih dan 20% data uji. serta, melakukan label encoding untuk target

In [133]:
X = gabungan.drop(columns=['PTN'])  
y = gabungan['PTN']

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Encode label string jadi angka
le = LabelEncoder()
y = le.fit_transform(y)

# Split: 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [134]:
y.shape

(3747,)

melakukan modeling

Model ini adalah jaringan saraf tiruan (neural network) yang dibangun menggunakan TensorFlow/Keras untuk tugas klasifikasi multi-kelas dengan 50 kelas. Berikut adalah detailnya:

1. **Arsitektur Model**:
   - **Input Layer**: Model menerima input dengan bentuk `(4,)`, yang berarti setiap sampel memiliki 4 fitur.
   - **Hidden Layer 1**: Lapisan Dense dengan 64 unit, menggunakan aktivasi 'relu' untuk memperkenalkan non-linearitas. Regularisasi L2 dengan faktor 0.01 ditambahkan untuk mencegah overfitting.
   - **BatchNormalization**: Diterapkan setelah lapisan pertama untuk menormalkan aktivasi, meningkatkan stabilitas dan kecepatan pelatihan.
   - **Dropout (0.4)**: 40% unit dinonaktifkan secara acak selama pelatihan untuk mengurangi overfitting.
   - **Hidden Layer 2**: Lapisan Dense dengan 32 unit, juga menggunakan 'relu' dan regularisasi L2 (0.01).
   - **BatchNormalization**: Diterapkan lagi untuk menjaga stabilitas.
   - **Dropout (0.3)**: 30% unit dinonaktifkan untuk lebih lanjut mengontrol overfitting.
   - **Output Layer**: Lapisan Dense dengan 50 unit dan aktivasi 'softmax', yang menghasilkan probabilitas untuk 50 kelas (jumlah total probabilitas = 1).

2. **Kompilasi Model**:
   - **Optimizer**: Adam dengan learning rate awal 0.001, cocok untuk optimasi efisien.
   - **Loss Function**: 'sparse_categorical_crossentropy', digunakan untuk klasifikasi multi-kelas dengan label integer (0 hingga 49).
   - **Metrics**: 'accuracy', untuk memantau akurasi selama pelatihan.

3. **Callback**:
   - **ReduceLROnPlateau**: Mengurangi learning rate sebesar faktor 0.5 jika 'val_loss' tidak membaik selama 5 epoch, dengan batas minimum 0.00001. Ini membantu model belajar lebih halus saat performa stagnan.

4. **Ringkasan Model**:
   - Fungsi `model.summary()` akan menampilkan detail arsitektur, termasuk jumlah parameter di setiap lapisan, yang berguna untuk memverifikasi desain model.

Model ini dirancang untuk tugas klasifikasi dengan dataset yang memiliki 4 fitur dan 50 kelas, dengan strategi untuk mencegah overfitting melalui regularisasi, batch normalization, dan dropout.

In [135]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.utils import class_weight

# Definisikan model
model = Sequential([
    Dense(64, activation='relu', input_shape=(4,), kernel_regularizer=tf.keras.regularizers.l2(0.01)),  # Kurangi unit, tambah L2
    BatchNormalization(),  # Stabilisasi pelatihan
    Dropout(0.4),  # Tingkatkan dropout
    Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(50, activation='softmax')  # 50 kelas
])


model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',  # Untuk label integer
    metrics=['accuracy']
)

model.summary()

# Callback untuk mengurangi learning rate saat performa stagnan
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)


c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                │ (None, 64)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 50)             │         1,650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,434 (17.32 KB)

 Trainable params: 4,242 (16.57 KB)

 Non-trainable params: 192 (768.00 B)

Kode ini digunakan untuk melatih model neural network yang telah didefinisikan sebelumnya menggunakan metode `fit` dari TensorFlow/Keras. Berikut detailnya:

1. **Data Pelatihan**:
   - `X_train`: Data fitur pelatihan dengan bentuk yang sesuai dengan `input_shape=(4,)` dari model.
   - `y_train`: Label pelatihan dalam format integer (0 hingga 49) untuk 50 kelas, sesuai dengan fungsi loss `sparse_categorical_crossentropy`.

2. **Parameter Pelatihan**:
   - `epochs=50`: Model akan dilatih selama 50 iterasi penuh melalui dataset, meningkatkan peluang konvergensi yang lebih baik.
   - `batch_size=32`: Data akan diproses dalam batch berukuran 32 sampel per iterasi, menyeimbangkan kecepatan dan stabilitas pelatihan.
   - `validation_split=0.2`: 20% dari data pelatihan akan dipisahkan sebagai data validasi untuk memantau performa model selama pelatihan.

3. **Callback**:
   - `lr_scheduler`: Callback `ReduceLROnPlateau` yang telah didefinisikan sebelumnya akan digunakan. Learning rate akan dikurangi sebesar faktor 0.5 jika `val_loss` tidak membaik selama 5 epoch, dengan batas minimum 0.00001, untuk mengoptimalkan proses pembelajaran.

4. **Proses**:
   - Model akan menyesuaikan bobotnya berdasarkan data pelatihan, dengan metrik akurasi dan loss yang dipantau pada data validasi. Hasil pelatihan akan mencakup riwayat loss dan akurasi yang dapat dianalisis untuk mengevaluasi performa.

Kode ini dirancang untuk melatih model dengan pendekatan yang mempertimbangkan konvergensi yang baik dan pencegahan overfitting melalui validasi dan penyesuaian learning rate.

In [136]:
# Latih model
model.fit(
    X_train, y_train,
    epochs=50,  # Tambah epoch untuk konvergensi lebih baik
    batch_size=32,
    validation_split=0.2,
    callbacks=[lr_scheduler]
)


Epoch 1/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.0484 - loss: 4.6723 - val_accuracy: 0.1267 - val_loss: 5.8331 - learning_rate: 0.0010
Epoch 2/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2633 - loss: 3.6129 - val_accuracy: 0.1700 - val_loss: 4.2896 - learning_rate: 0.0010
Epoch 3/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3358 - loss: 3.0486 - val_accuracy: 0.3900 - val_loss: 2.8598 - learning_rate: 0.0010
Epoch 4/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3464 - loss: 2.8620 - val_accuracy: 0.4350 - val_loss: 2.3476 - learning_rate: 0.0010
Epoch 5/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3917 - loss: 2.6355 - val_accuracy: 0.3767 - val_loss: 2.3759 - learning_rate: 0.0010
Epoch 6/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3897 - loss: 2.4705 - val_accuracy: 0.3817 - val_loss: 2.2965 - learning_rate: 0.0010
Epoch 7/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4167 - loss: 2.3294 - val_accuracy: 

In [137]:
y_pred = model.predict(X_test)
y_pred = le.inverse_transform(np.argmax(y_pred, axis=1))
y_test = le.inverse_transform(y_test)

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"\nAccuracy Score: {accuracy*100:.0f}%")
print(f"F1 Score: {f1*100:.0f}")

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Confusion Matrix:
[[ 2  0  2 ...  0  0  0]
 [ 0 15  0 ...  0  0  0]
 [ 0  3  2 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 1  0  0 ...  0  0  0]]

Classification Report:
                                     precision    recall  f1-score   support

           INSTITUT PERTANIAN BOGOR       0.09      0.18      0.12        11
         INSTITUT TEKNOLOGI BANDUNG       0.33      0.41      0.37        37
INSTITUT TEKNOLOGI SEPULUH NOPEMBER       0.14      0.07      0.10        28
              UNIVERSITAS AIRLANGGA       0.78      0.90      0.83       103
                UNIVERSITAS ANDALAS       0.00      0.00      0.00         3
              UNIVERSITAS BRAWIJAYA       0.33      0.70      0.45        76
           UNIVERSITAS CENDERAWASIH       0.00      0.00      0.00         2
             UNIVERSITAS DIPONEGORO       0.00      0.00      0.00        22
            UNIVERSITAS GADJAH MADA       0.58      0.36      0.44   

c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

akurasi pada model adalah 56% angka ini mungkin cukup rendah, tetapi dapat ditoleransi karena beberapa perguruan tinggi memiliki kesamaan pola sehingga sulit untuk membedakan secara pasti berdasarkan fitur yang digunakan. Dengan akurasi 56%, model masih dapat dianggap berguna untuk analisis awal atau sebagai panduan, terutama jika data tambahan atau fitur yang lebih spesifik dapat diterapkan untuk meningkatkan pemisahan antar kelas di masa depan.

In [138]:
y_pred

array(['UNIVERSITAS INDONESIA', 'UNIVERSITAS JENDERAL SOEDIRMAN',
       'INSTITUT PERTANIAN BOGOR', 'UNIVERSITAS JENDERAL SOEDIRMAN',
       'UNIVERSITAS JENDERAL SOEDIRMAN', 'UNIVERSITAS BRAWIJAYA',
       'UNIVERSITAS NEGERI SEMARANG', 'UNIVERSITAS BRAWIJAYA',
       'UNIVERSITAS BRAWIJAYA', 'UNIVERSITAS INDONESIA',
       'UNIVERSITAS GADJAH MADA', 'UNIVERSITAS BRAWIJAYA',
       'UNIVERSITAS BRAWIJAYA', 'UNIVERSITAS AIRLANGGA',
       'INSTITUT PERTANIAN BOGOR', 'UNIVERSITAS AIRLANGGA',
       'UNIVERSITAS BRAWIJAYA', 'UNIVERSITAS JENDERAL SOEDIRMAN',
       'UNIVERSITAS AIRLANGGA', 'UNIVERSITAS AIRLANGGA',
       'UNIVERSITAS GADJAH MADA', 'UNIVERSITAS INDONESIA',
       'UNIVERSITAS GADJAH MADA', 'INSTITUT TEKNOLOGI BANDUNG',
       'UNIVERSITAS AIRLANGGA', 'UNIVERSITAS NEGERI SEMARANG',
       'INSTITUT TEKNOLOGI SEPULUH NOPEMBER', 'UNIVERSITAS BRAWIJAYA',
       'UNIVERSITAS INDONESIA', 'UNIVERSITAS BRAWIJAYA',
       'UNIVERSITAS AIRLANGGA', 'UNIVERSITAS JENDERAL SOEDIRMAN',


In [139]:
import joblib

joblib.dump(model, 'model_klasifikasi.pkl')

['model_klasifikasi.pkl']

In [140]:
joblib.dump(le, 'encoder.joblib')

['encoder.joblib']